In [2]:
import numpy as np
import json
import pandas as pd
import recordlinkage
import re

# read data

In [65]:
#with open('/home/p/wines/winerecommender/data-matching/vivino_data_combined.json') as f:
#    vivino_data = json.load(f)
vivino_full = pd.read_csv('/home/p/wines/winerecommender/data-matching/vivino_full.csv')
di = pd.read_excel('/home/p/wines/winerecommender/Scraping/Data Sources - scraped/Direkt-Import.xlsx')
dv = pd.read_excel('/home/p/wines/winerecommender/Scraping/Data Sources - scraped/Divino.xlsx')
jacques = pd.read_csv('/home/p/wines/winerecommender/Scraping/jacques.de/jacques_wines.csv', encoding='utf-8')

#final = pd.read_csv('/home/p/wines/winerecommender/data-matching/matching_table.csv')


# format vivino data after extraction if necessary

In [ ]:
# # filter vivino data by keys used for matching, transform to DataFrame and export as .csv
# cols = ['wine_id','wine_name','wine_winery', 'wine_country', 'wine_region', 'wine_year', 'wine_type', 'wine_price', 'wine_alcohol']

# vivDict_full = [{k: dic[k] for k in vivino_data[0].keys()} for dic in vivino_data]
# vivino_df = pd.DataFrame.from_dict(vivDict_full)
# vivino_df = vivino_df.set_index('wine_id')
# vivino_df.to_csv('/home/p/wines/winerecommender/data-matching/vivino_full.csv')

# vivino_sub = vivino_df[cols]
# vivino_sub = vivino_sub.set_index('wine_id')
# vivino_sub.to_csv('/home/p/wines/winerecommender/data-matching/matching_vivino.csv')

# Define helper functions to format local wine data

In [124]:
#used to match dataframes by relevant columns

def updateWineType(t):
    switch = {
        'Rotwein': 'red',
        'Weißwein': 'white',
        'Weisswein': 'white',
        'Roséwein': 'rosé',
        'red': 'red',
        'white': 'white',
        'rosé': 'rosé'
    }

    try: 
        return switch.get(t.replace(',','').strip().split(' ')[0], 'sparkling')
    except:
        return 


#formats price strings to xx.xx instead of eg. xx,xx$ or $xx.xx. likely unneccessarily complicated but works for now
def formatPrice(x):
    x = str(x).strip()
    if re.match('/^\d+,\d+/', x):
        return float(x.replace('.','').replace(',', '.'))
    else:
        if re.match('/^\d+.\d+$/', x):
            return float(x)
        else:
            if re.match('^\s*$', x):
                return float(0)
            else:
                return float(x.replace('.','').replace(',', '.').replace('€', '').strip())

#removes years from the name, keeps the producer though
def removeYear(x):
    sub = re.sub('\d{4}', '', x)
    return sub.strip()

#formats alcohol percentage

def formatAlcohol(x):
    #if multiple percentages were scraped, just take the first one
    x = str(x).split(', ')[0]
    x = float(x.replace('%', '').replace(',','.').strip())
    return x

#takes year from name string of DirektImport wines

def extractYear(names):
    out = []
    for x in names:
        x= str(x)
        m = re.search('\d{4}', x)
        if m:
            out.append(m.group(0))
        else:
            out.append(None)
    return out

#split the string by triple whitespace, then split first element by comma. return first element of that as winery, second element of first split as wine. Works for Direktimport due to consistent naming

def splitWinery(names):
    wineries = []
    wines = [] 

    for x in names:
        w = str(x).strip().split('   ')
        wineries.append(w[0].strip().split(',')[0])
        wines.append(w[1].strip())
    return wineries, wines
#get quality indicators from table
def getQualityIndicators(array):
    seen = set()
    quals_set = []
    for item in array:
        quals = str(item).strip().split(',')
        for qual in quals:
            qual = qual.strip()
            if qual not in seen:
                seen.add(qual)
                quals_set.append(qual)
    return sorted(quals_set)


#remove winery, strip spaces, remove quality sigs
def formatName(x, quals):
    x = removeYear(str(x))
    for qual in quals:
        if re.match(qual, x):
            x = re.sub(qual, '', x)
    re.sub('\s+', ' ', x)
    return x

# replace values in df[col] with values in val if empty/None
def replaceMissingYears(df, col, val):
    indicies_of_missing = df[df[col].isnull()].index 
    for fill_index, dataframe_index in enumerate(indicies_of_missing):
        df.loc[dataframe_index, col] = val[dataframe_index]
    return df

# Format Local Wine Data

In [ ]:

# ## DO NOT USE AGAIN, extracted regions and formatted alcohol from jacques scraper

# def getInfoJacques(series):

#     regions = []
#     for item in series:
#         temp = re.sub(' +', ' ',item.replace("\xa0", "").strip()).split('\n\n')
#         try:
#             region.append(temp[0].split(', ')[1].strip())
#         except:
#             region.append('')
#     return region

# # jacques["Region"]=getInfoJacques(jacques.Name)

# # print(jacques["Region"])


# # jac2 = pd.read_csv('/home/p/wines/winerecommender/Scraping/jacques.de/jacques_wines.csv')

# # jac2['wine_region'] = jacques['Region']
# # jac2['wine_alcohol'] = jac2['wine_alcohol'].map(lambda x: x.replace('%',''))

# #jacques.to_csv('/home/p/wines/winerecommender/Scraping/jacques.de/jacques_wines.csv')

In [132]:
print(di.columns)
cols=['wine_name','wine_winery', 'wine_country', 'wine_region', 'wine_year', 'wine_type', 'wine_price', 'wine_alcohol', 'wine_thumb', 'wine_url', 'wine_seller']
di = pd.read_excel('/home/p/wines/winerecommender/Scraping/Data Sources - scraped/Direkt-Import.xlsx')
matching_df = pd.DataFrame(columns=cols)
di['Region:'] = di['Region:'].map(lambda x: str(x).strip().split(', ')[0])
di['Wein:'] = di['Wein:'].map(lambda x: updateWineType(x))
di['Alkohol in %:'] = di['Alkohol in %:'].map(lambda x: formatAlcohol(x))
di['Preis:'] = di['Preis:'].map(lambda x: formatPrice(x))
years_temp = extractYear(di['Name:'])
di['Name:'] = di['Name:'].map(lambda x: removeYear(x))

#not sure how to put this in a function
di = replaceMissingYears(di, 'Jahrgang:', years_temp)

# this is likely overkill and will produce weird results
# quals = getQualityIndicators(di['Qualität:'])
quals = []

di['Hersteller:'], di['Name:'] = splitWinery(di['Name:'])
di['Name:'] = di['Name:'].map(lambda x: formatName(x, quals))

di_new = di[['Name:', 'Hersteller:', 'Erzeugnis aus:', 'Region:', 'Jahrgang:', 'Wein:', 'Preis:', 'Alkohol in %:', 'Image:', 'Details-URL:', 'Seller']]
di_new.columns = cols

#remove sets of wines offered by direktimport
di_new = di_new[~di_new['wine_name'].str.contains('paket')]
di_new = di_new[~di_new['wine_name'].str.contains('Paket')]

di_new.to_csv('/home/p/wines/winerecommender/data-matching/direktimport_formatted.csv')
   

Index(['Id', 'Name:', 'Hersteller:', 'Erzeugnis aus:', 'Region:', 'Wein:',
       'Jahrgang:', 'Rebsorte:', 'Qualität:', 'Alkohol in %:', 'Geschmack:',
       'Allergene:', 'Trinktemperatur in °C:', 'Trinkempfehlung:',
       'Dekantieren:', 'Verschluß:', 'Säure g/l:', 'Restzucker g/l:', 'Boden:',
       'Essen:', 'Ausbauart:', 'Auszeichnungen:', 'Paßt zu:', 'Lage:',
       'Herstellung:', 'Produkt aus biologischem Anbau:', 'Biokontrollstelle:',
       'Diverses:', 'Zutaten:', 'Preis:', 'Image:', 'Details-URL:',
       'Verkostungsnotiz:', 'Seller'],
      dtype='object')


In [115]:
print(dv.columns)
dv = pd.read_excel('/home/p/wines/winerecommender/Scraping/Data Sources - scraped/Divino.xlsx')

#missing alcohol, reliable winery info

cols=['wine_name','wine_country', 'wine_region', 'wine_year', 'wine_type', 'wine_price', 'wine_thumb', 'wine_url', 'wine_seller']

years_temp = extractYear(dv['Name'])
dv = replaceMissingYears(dv, 'Jahrgang', years_temp)
dv['Art'] = dv['Art'].map(lambda x: updateWineType(x))
dv['Preis'] = dv['Preis'].map(lambda x: formatPrice(x))
dv['Name'] = dv['Name'].map(lambda x: removeYear(x))

dv_new = dv[['Name', 'Herkunftsland', 'Region', 'Jahrgang', 'Art', 'Preis', 'Image', 'Details-URL', 'Seller']]
dv_new.columns = cols

#house brand again
dv_new = dv_new[~dv_new['wine_name'].str.contains('divino.de')]

dv_new.to_csv('/home/p/wines/winerecommender/data-matching/divino_formatted.csv')



Index(['Id', 'Name', 'Jahrgang', 'Rebsorte', 'Weingut', 'Produktbezeichnung',
       'Ort', 'Passt gut zu', 'Herkunftsland', 'Anlass', 'Größe des Winzers',
       'Winzer', 'Region', 'Herstellungsmethode', 'Art', 'Lagerfähigkeit',
       'Charakteristik', 'Preis', 'Image', 'Details-URL', 'Beschreibung',
       'Seller'],
      dtype='object')


In [130]:
# jacques_new=jacques[['wine_name', 'wine_winery', 'wine_country', 'wine_region','wine_year', 'wine_type', 'wine_price', 'wine_alcohol','wine_seller']]
# matching_df= matching_df.append(jacques_new)

print(formatAlcohol(' 13,5 '))

13.5


In [ ]:
#exploration for wine region matching between vivino and local data

def getRegionSet(array):
    seen = set()
    regions = []
    for region in array:
        region = str(region).strip().split(', ')[0]
        if region not in seen:
            seen.add(region)
            regions.append(region)
    return sorted(regions)

regions = getRegionSet(final['wine_region'])

vivreg = getRegionSet([wine['wine_region'] for wine in vivino_data])

print('Matching: ',[region for region in regions if region in vivreg])
print('Missing: ',[region for region in regions if region not in vivreg])